In [2]:
import pandas as pd
import nltk
import twint
import twint.config
import numpy as np
import html
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
import string, re

tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'http\S+'
pat3 = r'pic.twitter.com\S+'
combined_pat = r'|'.join((pat1, pat2, pat3))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    lettersAndNumbers = re.sub("[^A-Za-z0-9]", " ", clean)
    lowerCase = lettersAndNumbers.lower()
    words = tok.tokenize(lowerCase)
    return (" ".join(words)).strip()

sid = SentimentIntensityAnalyzer()
phase1List = []
phase2List = []
phase3List = []
sentimentList = []
tweetList = []

movieDictionary = {}
data = pd.read_csv('MoviesKeyword.csv')
movieDictionary = {col: list(data[col]) for col in data.columns}
for movie in movieDictionary:
    c = twint.Config()
    c.Lang = 'en'
    c.Search = movieDictionary[movie][0]
    c.Format = "{id}|{date}|{time}|{tweet}|{likes_count}|{retweets_count}|{hashtags}"
    c.Since = movieDictionary[movie][1]
    c.Until = movieDictionary[movie][6]
    c.Store_csv = True
    tweetFile = movie + '.csv'
    c.Output = tweetFile
    twint.run.Search(c)
    movieDF = pd.read_csv(tweetFile, sep = '|')
    movieDF['date'] = pd.to_datetime(movieDF['date'], format = '%Y-%m-%d')
    tweets = alpha['tweet'].values.tolist()
    for t in tweets:
        cleanedTweet = tweet_cleaner(t)
        tweetList.append(cleanedTweet)
        sentimentScores = sid.polarity_scores(cleanedTweet)
        sentimentList.append(sentimentScores['compound'])
    movieDF['sentiment_polarity'] = sentimentList  
    movieDF = movieDF.sort_values(by=['date'], ascending = True)
    movieDF = movieDF.set_index(['date'])
    movieByDay = movieDF.resample('D').agg({'sentiment_polarity':'mean','tweet':'count','likes_count':'sum','retweets_count':'sum'})
    phase1Aggs = movieDF.loc['2016-06-23':'2016-07-02'].agg({'sentiment_polarity':'mean','tweet':'count','likes_count':'sum','retweets_count':'sum'})
    phase2Aggs = movieDF.loc['2016-07-03':'2016-07-12'].agg({'sentiment_polarity':'mean','tweet':'count','likes_count':'sum','retweets_count':'sum'})
    phase3Aggs = movieDF.loc['2013-07-13':'2016-07-23'].agg({'sentiment_polarity':'mean','tweet':'count','likes_count':'sum','retweets_count':'sum'})
    phase1AggsList = phase1Aggs.values.tolist()
    phase1List.append(phase1AggsList)
    phase2AggsList = phase2Aggs.values.tolist()
    phase2List.append(phase2AggsList)
    phase3AggsList = phase3Aggs.values.tolist()
    phase3List.append(phase3AggsList)
    d = {'Movie': movie,'Phase1':phase1List, 'Phase2':phase2List, 'Phase3':phase3List}
    movieByPhase = pd.DataFrame(d).set_index(['Movie'])
    movieByDay.to_csv('daily_testing.csv', header = True)
    movieByPhase.to_csv('phase_testing.csv', mode = 'a', header = True)

RuntimeError: This event loop is already running

In [4]:
import twint
import twint.config

c = twint.Config()